In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import pickle
from pandas.errors import EmptyDataError
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = Path('/content/drive/MyDrive/SIAM/validation 1')
cols = ['Некачественное ГДИС',
     'Влияние ствола скважины',
     'Радиальный режим',
     'Линейный режим',
     'Билинейный режим',
     'Сферический режим',
     'Граница постоянного давления',
     'Граница непроницаемый разлом',
     'Влияние ствола скважины_details',
     'Радиальный режим_details',
     'Линейный режим_details',
     'Билинейный режим_details',
     'Сферический режим_details',
     'Граница постоянного давления_details',
     'Граница непроницаемый разлом_details',
     'Время (часы)',
     'ΔP (атм)',
     'P′ (атм)',
     'file_name']

dict_valid = {key: [] for key in cols}

with open('/content/drive/MyDrive/SIAM/Сlassificator.pkl', "rb") as f2:                 # Открытие файла для модели классификации
    Classificator = pickle.load(f2)

with open('/content/drive/MyDrive/SIAM/Regresor.pkl', "rb") as f3:                 # Открытие файла для модели регрессии
    Regressor = pickle.load(f3)


k = 1
for file in folder_path.iterdir():
    with open(file, 'r') as f1:
        values = np.loadtxt(f1)
        df_classificator = np.mean(values, axis=0)
        if (df_classificator.size != 3):
            try:
                df_classificator = pd.read_table(file, header=None).to_numpy().reshape(1, 3)
            except EmptyDataError:
                dict_deploy = {key: 0 for key in cols[:-1]}
                dict_deploy['file_name'] = file.name
                for key in dict_deploy:
                    if key in dict_valid:
                        dict_valid[key].append(dict_deploy[key])
                continue
        else:
            df_classificator = df_classificator.reshape(1, 3)
        df_classificator_for_reg = df_classificator

        val_clas = np.round(Classificator.predict(df_classificator))[0]
        dict_classificator = {k: v for k, v in zip(cols[:8], val_clas)}

        dict_classificator['Время (часы)'] = df_classificator_for_reg[0][0]
        dict_classificator['ΔP (атм)'] = df_classificator_for_reg[0][1]
        dict_classificator['P′ (атм)'] = df_classificator_for_reg[0][2]

        CL_df_pred = np.array(list(dict_classificator.values())).reshape(1, -1)

        val_reg = Regressor.predict(CL_df_pred)
        dict_regressor = {k: v for k, v in zip(cols[8:-4], val_reg[0])}
        dict_regressor['file_name'] = file.name

        dict_deploy = dict_classificator | dict_regressor

        for key in dict_deploy:
            if key in dict_valid:
                dict_valid[key].append(dict_deploy[key])

        print(f"{k}: {dict_deploy}\n")
        k+=1

1: {'Некачественное ГДИС': 1, 'Влияние ствола скважины': 0, 'Радиальный режим': 0, 'Линейный режим': 0, 'Билинейный режим': 0, 'Сферический режим': 0, 'Граница постоянного давления': 0, 'Граница непроницаемый разлом': 0, 'Время (часы)': 3.1295999397926733, 'ΔP (атм)': 7.018999086003531, 'P′ (атм)': 1.4479491155985387, 'Влияние ствола скважины_details': 0.0, 'Радиальный режим_details': 0.0, 'Линейный режим_details': 0.0, 'Билинейный режим_details': 0.0, 'Сферический режим_details': 0.0, 'Граница постоянного давления_details': 0.0, 'Граница непроницаемый разлом_details': 0.0, 'file_name': 'd44db3ee-a83e-4d3d-8228-622e31e2cc40'}

2: {'Некачественное ГДИС': 1, 'Влияние ствола скважины': 0, 'Радиальный режим': 0, 'Линейный режим': 0, 'Билинейный режим': 1, 'Сферический режим': 0, 'Граница постоянного давления': 0, 'Граница непроницаемый разлом': 0, 'Время (часы)': 38.46301234957236, 'ΔP (атм)': 6.6360795920280005, 'P′ (атм)': 1.8410214805370102, 'Влияние ствола скважины_details': 0.0, 'Ради

<ipython-input-2-994aadee5bad>:36: UserWarning: loadtxt: input contained no data: "<_io.TextIOWrapper name='/content/drive/MyDrive/SIAM/validation 1/846291a2-6475-47ac-8f10-08eac6b93fcb' mode='r' encoding='UTF-8'>"
  values = np.loadtxt(f1)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


144: {'Некачественное ГДИС': 0, 'Влияние ствола скважины': 1, 'Радиальный режим': 0, 'Линейный режим': 0, 'Билинейный режим': 1, 'Сферический режим': 0, 'Граница постоянного давления': 0, 'Граница непроницаемый разлом': 1, 'Время (часы)': 18.223579364991746, 'ΔP (атм)': 4.496272737118984, 'P′ (атм)': 1.2167026570833985, 'Влияние ствола скважины_details': 1.050992460726485, 'Радиальный режим_details': 0.0, 'Линейный режим_details': 0.0, 'Билинейный режим_details': -0.18190625534743587, 'Сферический режим_details': 0.0, 'Граница постоянного давления_details': 0.0, 'Граница непроницаемый разлом_details': 60.10791020495711, 'file_name': 'de590091-6525-4a30-a352-d39de05744fd'}

145: {'Некачественное ГДИС': 1, 'Влияние ствола скважины': 1, 'Радиальный режим': 0, 'Линейный режим': 1, 'Билинейный режим': 0, 'Сферический режим': 0, 'Граница постоянного давления': 0, 'Граница непроницаемый разлом': 0, 'Время (часы)': 7.3965517241981455, 'ΔP (атм)': 101.98422104058342, 'P′ (атм)': 54.899420100399

In [3]:
result = pd.DataFrame(dict_valid, columns=cols)
result = result.drop(columns=['Время (часы)', 'ΔP (атм)', 'P′ (атм)'])
Cols = result.columns.to_list()
new_order = [Cols[-1]] + Cols[:-1]
result = result[new_order]

In [ ]:
exe = pd.read_csv('submit1.csv')

In [6]:
true_orders = exe['file_name'].to_list()

In [ ]:
result["file_name"] = pd.Categorical(
    result["file_name"],
    categories=true_orders,
    ordered=True
)

df_sorted = result.sort_values("file_name").reset_index(drop=True)
df_sorted.to_csv('validation.csv', index=False)

In [9]:
result_order = df_sorted['file_name'].to_list()
k = 0
for i in range(len(true_orders)):
    if(true_orders[i] == result_order[i]):
        k+=1
print(k)

500
